In [20]:
# https://www.tensorflow.org/extend/estimators
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# tensorflow
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.learn as tflearn
import tensorflow.contrib.layers as tflayers

# keras
from tensorflow.contrib.keras.python.keras.layers import Dense, LSTM, GRU, Activation
from tensorflow.contrib.keras.python.keras.utils.data_utils import get_file

# input data
from tensorflow.examples.tutorials.mnist import input_data

# estimators
from tensorflow.contrib import learn

# estimator "builder"
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

# helpers
import numpy as np
import random
import sys

# enable logs
tf.logging.set_verbosity(tf.logging.INFO)

def sample(preds, temperature=1.0):
    #print(preds)
    return np.argmax(preds)

# THE MODEL
def model_fn(features, targets, mode, params):
    """Model function for Estimator."""
    
    # 1. Configure the model via TensorFlow operations
    # First, build all the model, a good idea is using Keras or tf.layers
    # since these are high-level API's
    #lstm = GRU(128, input_shape=(params["maxlen"], params["vocab_size"]))(features)
    #preds = Dense(params["vocab_size"], activation='sigmoid')(lstm)
    
    # 0. Reformat input shape to become a sequence
    lstm1 = GRU(128, input_shape=(params["maxlen"], params["vocab_size"]),
                return_sequences=True)(features)
    lstm2 = GRU(128)(lstm1)
    preds = Dense(params["vocab_size"])(lstm2)
    preds_softmax = Activation("softmax")(preds)

    # 2. Define the loss function for training/evaluation
    loss = None
    train_op = None
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    if mode != learn.ModeKeys.INFER:
        loss = tf.losses.softmax_cross_entropy(
            onehot_labels=targets, logits=preds)

    # 3. Define the training operation/optimizer
    
    # Configure the Training Op (for TRAIN mode)
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=params["learning_rate"],
            optimizer="Adam",
        )

    # 4. Generate predictions
    predictions_dict = {
      "preds": preds_softmax
    }
    
    # 5. Define how you want to evaluate the model
    metrics = {
        "accuracy": tf.metrics.accuracy(tf.argmax(input=preds_softmax, axis=1), tf.argmax(input=targets, axis=1))
    }
    
    # 6. Return predictions/loss/train_op/eval_metric_ops in ModelFnOps object
    return model_fn_lib.ModelFnOps(
      mode=mode,
      predictions=predictions_dict,
      loss=loss,
      train_op=train_op,
      eval_metric_ops=metrics)

In [2]:
print('Getting data')

#path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
path = 'shakespeare.txt'
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 1
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.float32)
y = np.zeros((len(sentences), len(chars)), dtype=np.float32)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print(X[0])

Getting data
corpus length: 1115394
total chars: 39
nb sequences: 1115354
Vectorization...
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [ ]:
# PARAMETERS
LEARNING_RATE = 0.0001
BATCH_SIZE = 64
STEPS = 10000

NUM_OUTPUTS_PRED = 40

# Set model params
model_params = {"learning_rate": LEARNING_RATE, "vocab_size": len(chars), "maxlen": maxlen}

# Instantiate Estimator
nn = tf.contrib.learn.Estimator(model_fn=model_fn, params=model_params)

# Score accuracy
for iteration in range(1, 600):
    print()
    print('-' * 50)
    print('Iteration', iteration)
  
    # Fit
    print('-' * 40)
    print("Training")
    print('-' * 40)
    nn.fit(x=X, y=y, steps=STEPS, batch_size=BATCH_SIZE)

# choose a random sentence
start_index = random.randint(0, len(text) - maxlen - 1)
sentence = text[start_index: start_index + maxlen]    
    
# generate output using the RNN model
original_sentence = sentence
generated = sentence
for i in range(NUM_OUTPUTS_PRED):
    x = np.zeros((1, maxlen, len(chars)), dtype=np.float32)
    for t, char in enumerate(sentence):
        x[0, t, char_indices[char]] = 1.

    p = None
    for e in nn.predict(x):
        if p is None: p = e["preds"]
    next_index = sample(p)
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char

print('\n' * 10, '-' * 100)
print('HERE')
print(generated)
print(original_sentence)
print('-' * 100, '\n' * 10)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_type': None, '_save_checkpoints_steps': None, '_model_dir': None, '_tf_random_seed': None, '_is_chief': True, '_task_id': 0, '_master': '', '_environment': 'local', '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_num_worker_replicas': 0, '_evaluation_master': '', '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6340404438>}

--------------------------------------------------
Iteration 1
----------------------------------------
Training
----------------------------------------
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example co

/usr/local/lib/python3.4/dist-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpv058jrrr/model.ckpt.
INFO:tensorflow:loss = 3.66103, step = 1
INFO:tensorflow:global_step/sec: 11.2895
INFO:tensorflow:loss = 3.37729, step = 101 (8.859 sec)
INFO:tensorflow:global_step/sec: 11.5884
INFO:tensorflow:loss = 3.15746, step = 201 (8.629 sec)
INFO:tensorflow:global_step/sec: 11.582
INFO:tensorflow:loss = 2.96604, step = 301 (8.634 sec)
INFO:tensorflow:global_step/sec: 11.2301
INFO:tensorflow:loss = 2.93641, step = 401 (8.905 sec)
INFO:tensorflow:global_step/sec: 11.5129
INFO:tensorflow:loss = 3.12586, step = 501 (8.686 sec)
INFO:tensorflow:global_step/sec: 11.6271
INFO:tensorflow:loss = 3.16031, step = 601 (8.601 sec)
INFO:tensorflow:global_step/sec: 11.7331
INFO:tensorflow:loss = 2.9931, step = 701 (8.523 sec)
INFO:tensorflow:global_step/sec: 11.6664
INFO:tensorflow:loss = 2.91062, step = 801 (8.572 sec)
INFO:tensorflow:global_step/sec: 11.7135
INFO:tensorflow:loss = 2.90223, s

INFO:tensorflow:loss = 2.10164, step = 8301 (8.957 sec)
INFO:tensorflow:global_step/sec: 11.1456
INFO:tensorflow:loss = 2.03846, step = 8401 (8.972 sec)
INFO:tensorflow:global_step/sec: 11.1988
INFO:tensorflow:loss = 2.17244, step = 8501 (8.930 sec)
INFO:tensorflow:global_step/sec: 10.9788
INFO:tensorflow:loss = 2.18528, step = 8601 (9.108 sec)
INFO:tensorflow:global_step/sec: 10.8119
INFO:tensorflow:loss = 2.23051, step = 8701 (9.249 sec)
INFO:tensorflow:global_step/sec: 10.9493
INFO:tensorflow:loss = 1.99174, step = 8801 (9.133 sec)
INFO:tensorflow:global_step/sec: 10.8323
INFO:tensorflow:loss = 2.05067, step = 8901 (9.232 sec)
INFO:tensorflow:global_step/sec: 10.7575
INFO:tensorflow:loss = 2.52981, step = 9001 (9.296 sec)
INFO:tensorflow:global_step/sec: 10.9257
INFO:tensorflow:loss = 2.00066, step = 9101 (9.153 sec)
INFO:tensorflow:global_step/sec: 10.9988
INFO:tensorflow:loss = 2.09248, step = 9201 (9.092 sec)
INFO:tensorflow:global_step/sec: 11.0873
INFO:tensorflow:loss = 2.4681,

In [4]:
# choose a random sentence
start_index = random.randint(0, len(text) - maxlen - 1)
sentence = text[start_index: start_index + maxlen]    
    
# generate output using the RNN model
original_sentence = sentence
generated = sentence
for i in range(NUM_OUTPUTS_PRED):
    x = np.zeros((1, maxlen, len(chars)), dtype=np.float32)
    for t, char in enumerate(sentence):
        x[0, t, char_indices[char]] = 1.

    p = None
    for e in nn.predict(x):
        if p is None: p = e["preds"]
    next_index = sample(p)
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char

print('\n' * 10, '-' * 100)
print('HERE')
print(generated)
print(original_sentence)
print('-' * 100, '\n' * 10)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


/usr/local/lib/python3.4/dist-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Restoring parameters from /tmp/tmpceqk5qg1/model.ckpt-19101
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from /tmp/tmpceqk5qg1/model.ckpt-19101
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from /tmp/tmpceqk5qg1/model.ckpt-19101
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
availab

INFO:tensorflow:Restoring parameters from /tmp/tmpceqk5qg1/model.ckpt-19101
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from /tmp/tmpceqk5qg1/model.ckpt-19101
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from /tmp/tmpceqk5qg1/model.ckpt-19101
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
availab

INFO:tensorflow:Restoring parameters from /tmp/tmpceqk5qg1/model.ckpt-19101
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from /tmp/tmpceqk5qg1/model.ckpt-19101
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from /tmp/tmpceqk5qg1/model.ckpt-19101
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
availab

In [10]:
# generate output using the RNN model
original_sentence = "time"
sentence = original_sentence
generated = sentence
for i in range(10):
    x = np.zeros((1, maxlen, len(chars)), dtype=np.float32)
    for t, char in enumerate(sentence):
        x[0, t, char_indices[char]] = 1.

    p = None
    for e in nn.predict(x):
        if p is None: p = e["preds"]
    
    print(p)
    next_index = sample(p)
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char

print('\n' * 10, '-' * 100)
print('HERE')
print(generated)
print(original_sentence)
print('-' * 100, '\n' * 10)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


/usr/local/lib/python3.4/dist-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Restoring parameters from /tmp/tmpceqk5qg1/model.ckpt-19101
[  5.10431186e-04   9.98117208e-01   5.90219262e-11   2.03930277e-08
   2.06349871e-08   2.60475685e-09   2.82078217e-05   3.51113055e-10
   3.47168294e-09   1.67235417e-08   3.75141553e-07   7.82775580e-12
   3.99932887e-09   9.54997086e-06   3.24845595e-09   8.95351988e-08
   7.73334730e-11   5.15784544e-04   1.12831700e-08   2.41030378e-08
   1.61963342e-07   1.10379597e-05   4.23501363e-14   3.85896364e-08
   6.51927166e-07   1.49332386e-08   2.17957226e-08   9.30234746e-05
   3.41106809e-12   5.81926339e-14   6.43336443e-06   1.41718772e-06
   4.23773326e-06   6.97380048e-04   1.07524281e-10   3.80223630e-09
   2.50599356e-16   3.96385121e-06   5.61318136e-13]
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  e

KeyboardInterrupt: 